In [1]:
import pandas as pd 
import numpy as np 
import mysql_auth 
from MySQLConnection import *
from matplotlib import pyplot as plt

In [2]:
input_df = pd.read_csv('input_df.csv')
display(input_df)

C:\Users\jyc13\anaconda3\envs\owl-data\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (6,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MatchId,num_map,Map,map_type,Section,RoundName,Timestamp,Team,Player,...,OffensiveAssists/s,RCP,FBValue,DeathRisk,TF_order,TF_winner,TF_duration,TF_RCP_sum,TF_RCP_weighted_sum,DominanceIndex
0,0,37147,1,Busan,CONTROL,1.0,Sanctuary,2021-05-02 23:20:23,Dallas Fuel,Doha,...,0.0,-1.75,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.195832
1,1,37147,1,Busan,CONTROL,1.0,Sanctuary,2021-05-02 23:20:23,Dallas Fuel,Fielder,...,0.0,-1.75,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.195832
2,2,37147,1,Busan,CONTROL,1.0,Sanctuary,2021-05-02 23:20:23,Dallas Fuel,Jecse,...,0.0,-1.75,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.195832
3,3,37147,1,Busan,CONTROL,1.0,Sanctuary,2021-05-02 23:20:23,Houston Outlaws,Crimzo,...,0.0,-1.75,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.195832
4,4,37147,1,Busan,CONTROL,1.0,Sanctuary,2021-05-02 23:20:23,Houston Outlaws,Happy,...,0.0,-1.75,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.195832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485317,2485317,37404,3,Eichenwalde,HYBRID,2.0,NaN,2021-06-06 19:59:43,Houston Outlaws,Dreamer,...,0.0,0.00,0.0,0.0,7.0,Dallas Fuel,66.927,34.833333,50.219882,-0.119838
2485318,2485318,37404,3,Eichenwalde,HYBRID,2.0,NaN,2021-06-06 19:59:43,Houston Outlaws,Happy,...,0.0,0.00,0.0,0.0,7.0,Dallas Fuel,66.927,34.833333,50.219882,-0.119838
2485319,2485319,37404,3,Eichenwalde,HYBRID,2.0,NaN,2021-06-06 19:59:43,Houston Outlaws,joobi,...,0.0,0.00,0.0,0.0,7.0,Dallas Fuel,66.927,34.833333,50.219882,-0.119838
2485320,2485320,37404,3,Eichenwalde,HYBRID,2.0,NaN,2021-06-06 19:59:43,Houston Outlaws,joobi,...,0.0,0.00,0.0,0.0,7.0,Dallas Fuel,66.927,34.833333,50.219882,-0.119838


In [3]:
input_df['position_x'] = input_df['position'].str.split(',').str[0].astype('float')
input_df['position_y'] = input_df['position'].str.split(',').str[2].astype('float')
input_df['position_z'] = input_df['position'].str.split(',').str[1].astype('float')

In [4]:
input_df[['position_x_norm', 'position_y_norm', 'position_z_norm']] = input_df.groupby(['MatchId', 'num_map', 'Section'])[['position_x', 'position_y', 'position_z']].apply(lambda x: (x - x.min()) / (x.max() - x.min()) )

In [5]:
avg_position_x = input_df.groupby(['MatchId', 'num_map', 'Section', 'Timestamp', 'Team'])[['position_x_norm']].mean()
avg_position_y = input_df.groupby(['MatchId', 'num_map', 'Section', 'Timestamp', 'Team'])[['position_y_norm']].mean()
avg_position_z = input_df.groupby(['MatchId', 'num_map', 'Section', 'Timestamp', 'Team'])[['position_z_norm']].mean()

input_df = input_df.set_index(['MatchId', 'num_map', 'Section', 'Timestamp', 'Team'])

df_merge = pd.merge(input_df, avg_position_x, left_index=True, right_index=True, suffixes=('', '_avg'))
df_merge = pd.merge(df_merge, avg_position_y, left_index=True, right_index=True, suffixes=('', '_avg'))
df_merge = pd.merge(df_merge, avg_position_z, left_index=True, right_index=True, suffixes=('', '_avg'))

df_merge.dropna(axis=0, how='any', inplace=True)

In [10]:
df_merge['distance_from_center'] = ((df_merge['position_x_norm_avg'] - df_merge['position_x_norm'])**2 + (df_merge['position_y_norm_avg'] - df_merge['position_y_norm'])**2 + (df_merge['position_z_norm_avg'] - df_merge['position_z_norm'])**2)**(1/2)
display(df_merge)

Unnamed: 0  \
MatchId num_map Section Timestamp           Team                          
37147   1       1.0     2021-05-02 23:20:54 Dallas Fuel             201   
                                            Dallas Fuel             203   
                                            Dallas Fuel             204   
                                            Dallas Fuel             205   
                                            Dallas Fuel             206   
...                                                                 ...   
37404   1       2.0     2021-06-06 19:14:10 Houston Outlaws     2473218   
                                            Houston Outlaws     2473221   
                                            Houston Outlaws     2473222   
                                            Houston Outlaws     2473225   
                                            Houston Outlaws     2473226   

                                                               Map map_type  \
MatchId num_map Section Timestamp           Team                              
37147   1       1.0     2021-05-02 23:20:54 Dallas Fuel      Busan  CONTROL   
                                            Dallas Fuel      Busan  CONTROL   
                                            Dallas Fuel      Busan  CONTROL   
                                            Dallas Fuel      Busan  CONTROL   
                                            Dallas Fuel      Busan  CONTROL   
...                                                            ...      ...   
37404   1       2.0     2021-06-06 19:14:10 Houston Outlaws  Busan  CONTROL   
                                            Houston Outlaws  Busan  CONTROL   
                                            Houston Outlaws  Busan  CONTROL   
                                            Houston Outlaws  Busan  CONTROL   
                                            Houston Outlaws  Busan  CONTROL   

                                                             RoundName  \
MatchId num_map Section Timestamp           Team                         
37147   1       1.0     2021-05-02 23:20:54 Dallas Fuel      Sanctuary   
                                            Dallas Fuel      Sanctuary   
                                            Dallas Fuel      Sanctuary   
                                            Dallas Fuel      Sanctuary   
                                            Dallas Fuel      Sanctuary   
...                                                                ...   
37404   1       2.0     2021-06-06 19:14:10 Houston Outlaws  Sanctuary   
                                            Houston Outlaws  Sanctuary   
                                            Houston Outlaws  Sanctuary   
                                            Houston Outlaws  Sanctuary   
                                            Houston Outlaws  Sanctuary   

                                                               Player  \
MatchId num_map Section Timestamp           Team                        
37147   1       1.0     2021-05-02 23:20:54 Dallas Fuel          Doha   
                                            Dallas Fuel      Fearless   
                                            Dallas Fuel       Fielder   
                                            Dallas Fuel        HanBin   
                                            Dallas Fuel         Jecse   
...                                                               ...   
37404   1       2.0     2021-06-06 19:14:10 Houston Outlaws    Danteh   
                                            Houston Outlaws   Dreamer   
                                            Houston Outlaws     Happy   
                                            Houston Outlaws     joobi   
                                            Houston Outlaws     piggy   

                                                                      Hero  \
MatchId num_map Section Timestamp           Team                  